In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import pairwise_distances

In [2]:
samples = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 50, 51, 52, 53, 54, 55, 57, 58, 61]

In [4]:
for sample_idx in samples:
    features = np.load(f'./valid_and_test_moth_classification_hmc_D2048_fg_shuffled_{sample_idx:02d}.npy', allow_pickle=True)
    y_valid= np.load(f'./datasplit_cache/y_valid_fg_shuffled_{sample_idx:02d}.npy', allow_pickle=True)
    y_test = np.load(f'./datasplit_cache/y_test_fg_shuffled_{sample_idx:02d}.npy', allow_pickle=True)

    data = np.concatenate([np.concatenate([y_valid, y_test])[:, 0:1], features], axis=1)
    max_fml_idx = np.unique(data[:, 0]).max()

    result = []
    for i in range(0, max_fml_idx.astype(int) + 1):
        fml_features = [e[1:] for e in data if e[0] == i]
        if fml_features:
            amount = len(fml_features)
            group_center = sum(fml_features) / amount            
            dist_mat = pairwise_distances(np.vstack([group_center, fml_features]))
            avg_dist_to_center = sum(dist_mat[0, 1:]) / amount
            center_loss = sum([e ** 2 for e in dist_mat[0, 1:]]) / 2
            result.append([i, amount, avg_dist_to_center, center_loss])
        else:
            result.append([i, np.nan, np.nan])

    df = pd.DataFrame(result, columns=['Family', 'Amount', 'Average distance', 'Center loss'])
    df.to_csv(f'./result/mean_distance_{sample_idx:02d}.csv', sep='\t', index=False)